In [3]:
import os
import cv2

In [1]:
# Set the paths for the input and output folders
input_dir = 'faceEmotionDetection/train'
output_dir = 'onlyFaces'

In [4]:
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Define a function to detect faces in an image and save them to a new folder
def extract_faces(input_path, output_path):
    # Load the image
    img = cv2.imread(input_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Load the face detection model
    caffeModelFile = "/content/res10_300x300_ssd_iter_140000.caffemodel"
    caffeProtoTxt = "/content/deploy.prototxt"
    caffeDetector = cv2.dnn.readNetFromCaffe(caffeProtoTxt, caffeModelFile)

    # Detect faces in the image
    (h, w) = img.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    caffeDetector.setInput(blob)
    detections = caffeDetector.forward()

    # Save each face as a new image in the output folder
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            face_img = img[startY:endY, startX:endX]
            
            # output_path is equal to train/classFolder
            # the second one is for saving multiple faces (we can have multiple faces in one image)
            # third is the image that we want to save
            
            cv2.imwrite(os.path.join(output_path, f'{os.path.splitext(os.path.basename(input_path))[0]}_{i}.jpg'), face_img)

In [ ]:
# we have main input_dir as train and validation one by one

# Loop over each class folder in the input directory
for class_folder in os.listdir(input_dir):
    
    # class_folder is equal to 9 Classes inside train and validation 
    
    # Create a new folder for the class in the output directory
    class_output_dir = os.path.join(output_dir, class_folder)
    os.makedirs(class_output_dir, exist_ok=True)

    # Loop over each image in the class folder
    for image_file in os.listdir(os.path.join(input_dir, class_folder)):
        
        # Extract faces from the image and save them to the class folder in the output directory
        input_path = os.path.join(input_dir, class_folder, image_file)
        
        extract_faces(input_path, class_output_dir)
